In [21]:
!pip install requests vaderSentiment python-dotenv sqlalchemy psycopg2-binary


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [22]:
import os
import requests
from datetime import datetime
from sqlalchemy import create_engine, text
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

# Load API key and DB URL
GNEWS_API_KEY = os.getenv("GNEWS_API_KEY")
DATABASE_URL = os.getenv("DATABASE_URL")

# Assert to ensure the keys are loaded properly
assert GNEWS_API_KEY is not None, "GNEWS_API_KEY not loaded. Check your .env file."
assert DATABASE_URL is not None, "DATABASE_URL not loaded. Check your .env file."

# Setup SQLAlchemy engine and VADER
engine = create_engine(DATABASE_URL)
vader = SentimentIntensityAnalyzer()


In [23]:
def analyze_sentiment(text):
    score = vader.polarity_scores(text)['compound']
    label = 'POS' if score > 0.05 else 'NEG' if score < -0.05 else 'NEU'
    return score, label, 'VADER', None

def insert_sentiment(data_dict):
    with engine.begin() as conn:
        query = text("""
            INSERT INTO sentiment_data (
                ticker, source, api_vendor, date, headline, sentiment_score,
                sentiment_label, article_url, author, platform_tags,
                model_used, confidence
            ) VALUES (
                :ticker, :source, :api_vendor, :date, :headline, :sentiment_score,
                :sentiment_label, :article_url, :author, :platform_tags,
                :model_used, :confidence
            )
        """)
        conn.execute(query, data_dict)


In [24]:
def fetch_gnews_sentiment(ticker):
    url = f"https://gnews.io/api/v4/search?q={ticker}&token={GNEWS_API_KEY}&lang=en&max=10"
    response = requests.get(url)

    if response.status_code != 200:
        raise Exception(f"GNews API failed: {response.status_code} - {response.text}")
    
    articles = response.json().get("articles", [])
    
    for article in articles:
        score, label, model, conf = analyze_sentiment(article["title"])
        data = {
            "ticker": ticker,
            "source": article.get("source", {}).get("name", "GNews"),
            "api_vendor": "GNews API",
            "date": datetime.strptime(article["publishedAt"][:10], "%Y-%m-%d").date(),
            "headline": article["title"],
            "sentiment_score": score,
            "sentiment_label": label,
            "article_url": article["url"],
            "author": article.get("author") or "Unknown",
            "platform_tags": "GNews",
            "model_used": model,
            "confidence": conf
        }
        insert_sentiment(data)
    
    print(f"Inserted sentiment records for {ticker} from GNews")


In [25]:
for ticker in ['NVDA', 'TSLA', 'AAPL']:
    fetch_gnews_sentiment(ticker)


Inserted sentiment records for NVDA from GNews
Inserted sentiment records for TSLA from GNews
Inserted sentiment records for AAPL from GNews
